### Loading libraries

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

### Loading the model

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# The name of the models to upload
models = [
    "lightblue/suzume-llama-3-8B-multilingual-orpo-borda-half",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/Qwen2-7B-Instruct-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
] # Try more models at https://huggingface.co/models
n=2
model_name = models[n].split('/')[1]
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models[n], # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

### Model Modification Parameters LoRA

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 1,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Preparing a dataset for training

### Post Generation

In [6]:
import requests
import os
import pandas as pd

# 10.5281/zenodo.17054610
record_id = '17054610'
filename_to_download = 'vk_posts_balanced.xlsx'

api_url = f'https://zenodo.org/api/records/{record_id}'

try:
    print(f"Searching for file '{filename_to_download}' in Zenodo repository with ID: {record_id}...")

    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()

    file_url = None
    for file in data.get('files', []):
        if file.get('key') == filename_to_download:
            file_url = file.get('links', {}).get('self')
            break

    if file_url:
        print(f"File '{filename_to_download}' found. Starting download...")

        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Save the file to the current directory
        with open(filename_to_download, 'wb') as f:
            for chunk in file_response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"File '{filename_to_download}' downloaded successfully.")
    else:
        print(f"File '{filename_to_download}' not found in the repository.")

except requests.exceptions.RequestException as e:
    print(f"An error occurred while accessing Zenodo: {e}")

posts = pd.read_excel('vk_posts_balanced.xlsx')

# Converting all columns to a string type
posts['Text'] = posts['text'].astype(str)

# Dictionary for replacing values
replacement_dict = {
    0: 'optional to nothing (0)',
    1: 'binding to discount (1)',
    2: 'obliging the group to make a gift (2)',
    3: 'obliging the group to give cashback (3)',
    4: 'obligating the group to deliver the goods on time (4)'
}
posts["Instruction"] = "Generate an advertising post for the food delivery group promising "
posts["Instruction"] = posts["Instruction"] + posts["class"].map(replacement_dict)
posts['Instruction'] = posts['Instruction'].astype(str)

posts["sys_promt"] = ("""You are an expert in generating VK group posts in the field of food delivery.
You will be given instructions for generating a group post.
Your task is to create a group post that follows the instructions."""
)
posts['sys_promt'] = posts['sys_promt'].astype(str)

# Replacing NaN values with empty strings
posts = posts.fillna('')

train = posts
train.head(3)

from datasets import Dataset
# Создание Dataset
ds = Dataset.from_dict({
    "output": posts['Text'],
    "input": posts['Instruction'],
    "instruction": posts['sys_promt']
})

ds[0]

Searching for file 'vk_posts_balanced.xlsx' in Zenodo repository with ID: 17054610...
File 'vk_posts_balanced.xlsx' found. Starting download...
File 'vk_posts_balanced.xlsx' downloaded successfully.


{'output': 'Розыгрыш 80 роллов - сет ONE LOVE \nОт тебя нужен только лайк💪\n\nДля участия в конкурсе: \n- Поставь лайк на пост и сториз\n- Напиши в комментариях твой любимый сет \n\nИтоги подведем в среду вечером, случайным образом.',
 'input': 'Generate an advertising post for the food delivery group promising obliging the group to make a gift (2)',
 'instruction': 'You are an expert in generating VK group posts in the field of food delivery. \nYou will be given instructions for generating a group post. \nYour task is to create a group post that follows the instructions.'}

### Converting a dataset to a suitable format

In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = ds
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

## Model training
Now we will use the `SFTTrainer` from Huggingface TRL! More documentation is here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer ). We do 60 steps for speed, but we can set `num_train_epochs=1` for a full run and disable `max_steps=None'. We also support the `DPOTrainer` from TRL!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2500 [00:00<?, ? examples/s]

In [9]:
#@title Indicators of available video memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.725 GB of memory reserved.


In [ ]:
os.environ["WANDB_DISABLED"] = "true"
trainer_stats = trainer.train()

In [ ]:
#@title Video memory usage indicators and statistics
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


## Запуск модели

### Generating posts

In [ ]:
import re
post_result=''
for Instruction in posts["Instruction"].unique():
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            posts["sys_promt"][0], # instruction
            Instruction, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    message_answer = tokenizer.batch_decode(outputs)[0].rsplit('Response:', 1)[-1]
    match = re.search(r'[012345]', str(Instruction))
    post_result+=str(int(match.group()))+' - '+str(message_answer)+'\n'
print(post_result)

In [ ]:

post_result=''
for Instruction in posts["Instruction"].unique():
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer in English\n "+str(posts["sys_promt"][0]), # instruction
            "Answer in English\n "+Instruction, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    message_answer = tokenizer.batch_decode(outputs)[0].rsplit('Response:', 1)[-1]
    match = re.search(r'[012345]', str(Instruction))
    post_result+=str(int(match.group()))+' - '+str(message_answer)+'\n'
print(post_result)

In [ ]:

my_file = open(f"{model_name}_post_gen.txt", "w+")
my_file.write(post_result)
my_file.close()